# Python 2 HSUTCC: Session 7: Special Task

## Special Task (Bonus 8% - Deadline Thursday 27 Nov 2025)

Symbolic differentiation allows to calculate a value of a derivative for an expression in a form of a sequence of symbols. Humans are likely to make mistakes during a long monotonous work and calculating of a direvative even not that hard but tidious process. Systems of symbolic calculations, like in this <a href='https://www.wolframalpha.com/input?i=derivate+x%5E3+++(1+-+x)+/+(1+-+x%5E2)'>example</a> are dedicated to save humanity from the need to spend their time for a potentially wrong calcuations.

We will try to implement a simple system of symbolic differentiation on Python. The main type in our system will be a class `Expr` – an expression. All successor of the class should implement:

- `__call__` method which calculates a value of an expression in a specified context: the context bounds names of variables in the expression with specific values.
- `d` (Leibniz's notation of derivative) method which takes a name of a variable wrt (from “with respect to”) and returns an expression for a derivative with respect to this variable.

In [ ]:
class Expr:
    def __call__(self, **context):
        pass

    def d(self, wrt):
        pass

**Important**: For each class, implement a `__str__` method which should return a corresponding formula in a form of S-expression.

1. Implement class for two types of expressions: `Const` – a constant and `Var` – a variable. For convinience next we will use not a constructors of classes but their one-letter synonyms:

```python
V = Var
C = Const
```

Example of class usage:

---

```python
C(42)() # print(C(42)) -> 42
```

```terminal
42
```

---

```python
C(42).d(V("x"))() # print(C(42).d(V("x"))) -> 0
```

```terminal
0
```

---

```python
V("x")(x=42) # print(V("x")) -> x
```

```terminal
42
```

---

```python
V("x").d(V("x"))() # print(V("x").d(V("x"))) -> 1
```

```terminal
1
```

---

```python
V("x").d(V("y"))() # print(V("x").d(V("x"))) -> 0
```

---

In [ ]:
# Your code here
class Const(Expr):
    def __init__(self, value):
        self.value = value

    def __call__(self, **context):
        return self.value

    def d(self, wrt):
        return Const(0)

    def __str__(self):
        return str(self.value)

class Var(Expr):
    def __init__(self, name):
        self.name = name

    def __call__(self, **context):
        return context[self.name]

    def d(self, wrt):
        # wrt can be a Var or a raw name
        name = wrt.name if isinstance(wrt, Var) else wrt
        return Const(1) if self.name == name else Const(0)

    def __str__(self):
        return str(self.name)

2. Implement classes for binary operations: `Sum`, `Product`, and `Fraction`. Binary operations by definition work with exactly two operands that’s why a constructor for all binary classes will be the same. It’s convinient to take it out into a separate base class:


In [ ]:
class BinOp(Expr):
    def __init__(self, expr1: Expr, expr2: Expr) -> None:
        self.expr1, self.expr2 = expr1, expr2

An example of using of some binary operations:

---

```python
# x + x^2
x = V("x")
Sum(x, Product(x, x)).d(x)(x=42) # (+ 1 (+ (* 1 x) (* x 1)))
```

```terminal
85
```

---

```python
Product(x, Sum(x, C(2)))(x=42) # (* x (+ x 2))
```

```terminal
1848
```

---

```python
Fraction(Product(x, V("y")), Sum(C(42), x)).d(x)(x=42, y=24)
```

```terminal
0.14285714285714285
```

---

```python
Fraction(Product(x, V("y")), Sum(C(42), x)).d(V("y"))(x=42, y=24)
```

```terminal
0.5
```

---

In [ ]:
# Your code here
class Sum(BinOp):
    def __call__(self, **context):
        return self.expr1(**context) + self.expr2(**context)

    def d(self, wrt):
        return Sum(self.expr1.d(wrt), self.expr2.d(wrt))

    def __str__(self):
        return "(+ {} {})".format(str(self.expr1), str(self.expr2))

class Product(BinOp):
    def __call__(self, **context):
        return self.expr1(**context) * self.expr2(**context)

    def d(self, wrt):
        # (u*v)' = u'*v + u*v'
        return Sum(Product(self.expr1.d(wrt), self.expr2), Product(self.expr1, self.expr2.d(wrt)))

    def __str__(self):
        return "(* {} {})".format(str(self.expr1), str(self.expr2))

class Fraction(BinOp):
    def __call__(self, **context):
        return self.expr1(**context) / self.expr2(**context)

    def d(self, wrt):
        # (u/v)' = (u'*v - u*v') / v^2
        num = Sum(Product(self.expr1.d(wrt), self.expr2), Product(Const(-1), Product(self.expr1, self.expr2.d(wrt))))
        den = Product(self.expr2, self.expr2)
        return Fraction(num, den)

    def __str__(self):
        return "(/ {} {})".format(str(self.expr1), str(self.expr2))

# Helper to ensure plain numbers become Const instances
def _ensure_expr(e):
    return e if isinstance(e, Expr) else Const(e)

# Monkey-patch operator overloads onto Expr so that expressions like
# V('x') + C(2) produce Sum/Prod/Fraction/Power nodes rather than numbers.
def _neg(self):
    return Product(Const(-1), self)

def _pos(self):
    return self

def _add(self, other):
    return Sum(self, _ensure_expr(other))

def _radd(self, other):
    return Sum(_ensure_expr(other), self)

def _sub(self, other):
    return Sum(self, Product(Const(-1), _ensure_expr(other)))

def _rsub(self, other):
    return Sum(_ensure_expr(other), Product(Const(-1), self))

def _mul(self, other):
    return Product(self, _ensure_expr(other))

def _rmul(self, other):
    return Product(_ensure_expr(other), self)

def _truediv(self, other):
    return Fraction(self, _ensure_expr(other))

def _rtruediv(self, other):
    return Fraction(_ensure_expr(other), self)

def _pow(self, other):
    return Power(self, _ensure_expr(other))

def _rpow(self, other):
    return Power(_ensure_expr(other), self)

# Attach to Expr so subclasses inherit these behaviors
Expr.__neg__ = _neg
Expr.__pos__ = _pos
Expr.__add__ = _add
Expr.__radd__ = _radd
Expr.__sub__ = _sub
Expr.__rsub__ = _rsub
Expr.__mul__ = _mul
Expr.__rmul__ = _rmul
Expr.__truediv__ = _truediv
Expr.__rtruediv__ = _rtruediv
Expr.__pow__ = _pow
Expr.__rpow__ = _rpow

# Implement Power behavior at runtime (Power class defined above)
def _power_call(self, **context):
    return self.expr1(**context) ** self.expr2(**context)

def _power_d(self, wrt):
    # assume exponent is Const for simplicity: (u^n)' = n*u^(n-1)*u'
    if isinstance(self.expr2, Const):
        n = self.expr2.value
        return Product(Product(Const(n), Power(self.expr1, Const(n - 1))), self.expr1.d(wrt))
    # General case (not required) - fallback to NotImplemented
    raise NotImplementedError('Power derivative only implemented for constant exponent')

def _power_str(self):
    return "(** {} {})".format(str(self.expr1), str(self.expr2))

Power.__call__ = _power_call
Power.d = _power_d
Power.__str__ = _power_str

# Convenient aliases
C = Const
V = Var

3. Implement a class `Power` for an operation of an exponentiation operation. For simplicity you can expect that a power – is an expression consisting only of constants.

---

```python
Power(Fraction(V("x"), C(4)), C(2))(x=42) # (** (/ x 4) 2)
```

```terminal
110.25
```

---

```python
Power(Fraction(V("x"), C(4)), C(2)).d(V("x"))(x=42)
```

```terminal
5.25
```

---

In [ ]:
class Power(BinOp):
    pass

4. Implement an overloading of arithmetic operators for a base class Expr. Overall there are needed to be implement 7 operators: 2 unary and 5 binary:

```terminal
-e          e.__neg__()
+e          e.__pos__()
e1 + e2     e1.__add__(e2)
e1 - e2     e1.__sub__(e2)
e1 * e2     e1.__mul__(e2)
e1 / e2     e1.__truediv__(e2)
e1 ** e2    e1.__pow__(e2)
```

---

```python
((C(1) - V("x")) ** C(3) + V("x"))(x=12)
```

```terminal
-1319
```

---

In [ ]:
class Expr():
    def __call__(self, **context):
        pass

    def d(self, wrt):
        pass

    # Overloading of arithmetic operators
    def __neg__(self):
        pass

In [ ]:
# To test your implementation, put all other classes here
# Restart the kernel, run the above and current cell, then do some testing

# Your code here
# Quick tests for implemented functionality
# Evaluate constants and variables
print('C(42)() ->', C(42)())
print('V(x)(x=42) ->', V('x')(x=42))
# Derivatives of simple expressions
x = V('x')
expr = Sum(x, Product(x, x))  # x + x^2
print('expr(x=42) ->', expr(x=42))
print('expr.d(x)(x=42) ->', expr.d(x)(x=42))
# Product and fraction
print('Product(x, Sum(x, C(2)))(x=42) ->', Product(x, Sum(x, C(2)))(x=42))
print('Fraction(Product(x, V(y)), Sum(C(42), x)).d(V(y))(x=42, y=24) ->', Fraction(Product(x, V('y')), Sum(C(42), x)).d(V('y'))(x=42, y=24))
# Power
print('Power(Fraction(V(x), C(4)), C(2))(x=42) ->', Power(Fraction(V('x'), C(4)), C(2))(x=42))
print('Power(Fraction(V(x), C(4)), C(2)).d(V(x))(x=42) ->', Power(Fraction(V('x'), C(4)), C(2)).d(V('x'))(x=42))
